In [1]:
import torch 
import argparse
import yaml
import time
import multiprocessing as mp
import torch.nn.functional as F
from tabulate import tabulate
from tqdm import tqdm
from torch.utils.data import DataLoader
from pathlib import Path
#from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DistributedSampler, RandomSampler
from torch import distributed as dist
from nmc.models import *
from nmc.datasets import * 
from nmc.augmentations import get_train_augmentation, get_val_augmentation
from nmc.losses import get_loss
from nmc.schedulers import get_scheduler
from nmc.optimizers import get_optimizer
from nmc.utils.utils import fix_seeds, setup_cudnn, cleanup_ddp, setup_ddp
from tools.val import evaluate_epi
from nmc.utils.episodic_utils import * 
from scipy.cluster import hierarchy
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from torchvision import models
import torch.nn as nn
from torch.optim import lr_scheduler
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mutual_info_score
from scipy.cluster import hierarchy
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss
from torch.utils.data import Dataset, DataLoader, Sampler


In [2]:
with open('configs/NMC.yaml') as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)
print(cfg)
fix_seeds(3407)
setup_cudnn()
gpu = setup_ddp()
save_dir = Path(cfg['SAVE_DIR'])
save_dir.mkdir(exist_ok=True)
cleanup_ddp()

{'DEVICE': 'cuda:1', 'SAVE_DIR': 'output', 'MODEL': {'NAME': 'FGMaxxVit', 'BACKBONE': 'FGMaxxVit', 'PRETRAINED': 'checkpoints/pretrained/maxvit_base_tf_512.in1k_pretrained_weights.pth', 'UNFREEZE': 'full', 'VERSION': 'ImageNet_APTOS_1024'}, 'DATASET': {'NAME': 'NMCDataset', 'ROOT': '/data/nmc/processed_image', 'TRAIN_RATIO': 0.7, 'VALID_RATIO': 0.15, 'TEST_RATIO': 0.15}, 'TRAIN': {'IMAGE_SIZE': [512, 512], 'BATCH_SIZE': 32, 'EPOCHS': 100, 'EVAL_INTERVAL': 25, 'AMP': False, 'DDP': False}, 'LOSS': {'NAME': 'BCEWithLogitsLoss', 'CLS_WEIGHTS': False}, 'OPTIMIZER': {'NAME': 'adamw', 'LR': 0.0001, 'WEIGHT_DECAY': 0.01}, 'SCHEDULER': {'NAME': 'warmuppolylr', 'POWER': 0.9, 'WARMUP': 10, 'WARMUP_RATIO': 0.1}, 'EVAL': {'MODEL_PATH': 'checkpoints/pretrained/FGMaxxVit/FGMaxxVit.FGMaxxVit.NMC.pth', 'IMAGE_SIZE': [512, 512]}, 'TEST': {'MODEL_PATH': 'checkpoints/pretrained/FGMaxxVit/FGMaxxVit.FGMaxxVit.NMC.pth', 'FILE': 'assests/ade', 'IMAGE_SIZE': [512, 512], 'OVERLAY': True}}


In [3]:
def extract_labels(dataloader):
    all_labels = []
    for _, labels in tqdm(dataloader, desc="Extracting labels"):
        all_labels.append(labels.numpy())
    return np.vstack(all_labels)
def calculate_mutual_info_matrix(label_matrix):
    n_labels = label_matrix.shape[1]
    mi_matrix = np.zeros((n_labels, n_labels))
    for i in range(n_labels):
        for j in range(i, n_labels):
            mi = mutual_info_score(label_matrix[:, i], label_matrix[:, j])
            mi_matrix[i, j] = mi
            mi_matrix[j, i] = mi
    return mi_matrix
def plot_heatmap(matrix, title):
    plt.figure(figsize=(10, 8))
    sns.heatmap(matrix, annot=False, cmap='coolwarm')
    plt.title(title)
    plt.show()
def analyze_trainloader_labels(trainloader):
    # 라벨 추출
    label_matrix = extract_labels(trainloader)

    # 상호 정보량 매트릭스 계산 및 시각화
    mi_matrix = calculate_mutual_info_matrix(label_matrix)
    plot_heatmap(mi_matrix, 'Label Mutual Information Matrix')
    return mi_matrix
def select_most_independent_labels(mi_matrix):
    linkage = hierarchy.linkage(mi_matrix, method='average')
    
    n_labels = mi_matrix.shape[0]
    ordered_labels = []
    remaining_clusters = list(range(2 * n_labels - 1))
    
    while len(ordered_labels) < n_labels:
        heights = [linkage[i-n_labels, 2] if i >= n_labels else 0 for i in remaining_clusters]
        max_height_idx = np.argmax(heights)
        selected_cluster = remaining_clusters[max_height_idx]
        
        if selected_cluster < n_labels:
            if selected_cluster not in ordered_labels:
                ordered_labels.append(selected_cluster)
            remaining_clusters.remove(selected_cluster)
        else:
            left, right = linkage[selected_cluster - n_labels, :2].astype(int)
            for child in [left, right]:
                if child < n_labels and child not in ordered_labels:
                    ordered_labels.append(child)
            remaining_clusters.remove(selected_cluster)
        
        # 모든 라벨이 선택되었다면 종료
        if len(ordered_labels) == n_labels:
            break
    
    return ordered_labels

In [4]:
start = time.time()
best_mf1 = 0.0
device = torch.device(cfg['DEVICE'])
print("device : ",device)
num_workers = mp.cpu_count()
train_cfg, eval_cfg = cfg['TRAIN'], cfg['EVAL']
dataset_cfg, model_cfg = cfg['DATASET'], cfg['MODEL']
loss_cfg, optim_cfg, sched_cfg = cfg['LOSS'], cfg['OPTIMIZER'], cfg['SCHEDULER']
epochs, lr = train_cfg['EPOCHS'], optim_cfg['LR']

image_dir = Path(dataset_cfg['ROOT']) / 'train_images'
transformations = get_train_augmentation(train_cfg['IMAGE_SIZE'])
batch_size = 32

dataset = eval(dataset_cfg['NAME'])(dataset_cfg['ROOT']+'/combined_images',dataset_cfg['TRAIN_RATIO'], dataset_cfg['VALID_RATIO'], dataset_cfg['TEST_RATIO'],  transformations)
trainset, valset, testset = dataset.get_splits()
# trainloader = DataLoader(trainset, batch_size=batch_size, num_workers=num_workers, drop_last=True, pin_memory=True)
valloader = DataLoader(valset, batch_size=1, num_workers=1, pin_memory=True)
testloader = DataLoader(testset, batch_size=1, num_workers=1, pin_memory=True)

device :  cuda:1
/data/nmc/processed_image/combined_images
(0,)             148
(0, 10)          110
(10,)             56
(0, 9, 10)        41
(0, 9)            41
                ... 
(1, 8, 9, 10)      1
(3, 4, 9, 10)      1
(5,)               1
(1, 2, 9, 10)      1
(3, 4)             1
Name: label, Length: 65, dtype: int64
train size: 686
(0,)                30
(0, 10)             25
(0, 9)              10
(0, 9, 10)           8
(9, 10)              5
(2, 10)              5
(1, 2)               4
(3, 10)              3
(1, 2, 3)            3
(1, 2, 3, 8)         3
(1, 10)              3
(8, 10)              2
(2, 8, 10)           2
(1, 2, 8, 10)        2
(0, 8)               2
(1, 2, 8, 9)         2
(1, 2, 3, 8, 10)     2
(1, 9, 10)           2
(0, 8, 10)           1
(1, 2, 5, 8)         1
(1, 2, 3, 9)         1
(1, 2, 9, 10)        1
(5, 8, 9, 10)        1
(2, 9, 10)           1
(2, 4)               1
(1, 2, 3, 4)         1
(3, 9)               1
(1, 4, 9, 10)        1
(2, 3, 9)   

In [5]:
len(trainset)

686

In [6]:
mi_matrix = analyze_trainloader_labels(trainloader)

NameError: name 'trainloader' is not defined

In [5]:
def calculate_metrics(all_preds, all_labels):
    # PyTorch 텐서를 CPU로 이동하고 NumPy 배열로 변환
    all_preds = all_preds.cpu().numpy()
    all_labels = all_labels.cpu().numpy()
    
    # 예측값을 0 또는 1로 변환 (임계값 0.5 사용)
    pred_binary = (all_preds > 0.5).astype(int)
    
    # 실제 데이터에 존재하는 클래스 인덱스 찾기
    active_classes = np.where(np.sum(all_labels, axis=0) > 0)[0]
    
    # 전체 정확도 계산
    accuracy = accuracy_score(all_labels, pred_binary)
    
    # 각 클래스별 F1 점수, 정밀도, 재현율 계산 (활성 클래스만)
    f1_scores = f1_score(all_labels[:, active_classes], pred_binary[:, active_classes], average=None, zero_division=0)
    precision_scores = precision_score(all_labels[:, active_classes], pred_binary[:, active_classes], average=None, zero_division=0)
    recall_scores = recall_score(all_labels[:, active_classes], pred_binary[:, active_classes], average=None, zero_division=0)
    
    # Hamming Loss 계산
    hamming = hamming_loss(all_labels, pred_binary)
    
    # 샘플당 정확히 모든 라벨을 맞춘 비율 (Subset Accuracy)
    subset_accuracy = (all_labels == pred_binary).all(axis=1).mean()
    
    return {
        'accuracy': accuracy,
        'hamming_loss': hamming,
        'subset_accuracy': subset_accuracy,
        'f1_scores': f1_scores,
        'precision_scores': precision_scores,
        'recall_scores': recall_scores,
        'active_classes': active_classes
    }

def print_metrics(metrics):
    print("=== Overall Metrics ===")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Hamming Loss: {metrics['hamming_loss']:.4f}")
    print(f"Subset Accuracy: {metrics['subset_accuracy']:.4f}")
    print("\n=== Class-wise Metrics ===")
    print("Class\t\tF1 Score\tPrecision\tRecall")
    print("-" * 50)
    for i, class_idx in enumerate(metrics['active_classes']):
        print(f"Class {class_idx}:\t{metrics['f1_scores'][i]:.4f}\t\t{metrics['precision_scores'][i]:.4f}\t\t{metrics['recall_scores'][i]:.4f}")
    
    print("\n=###############== Average Metrics (Active Classes) ==###############=")
    print(f"Macro F1: {np.mean(metrics['f1_scores']):.4f}")
    print(f"Macro Precision: {np.mean(metrics['precision_scores']):.4f}")
    print(f"Macro Recall: {np.mean(metrics['recall_scores']):.4f}")

In [6]:
class BalancedMultiLabelSampler(Sampler):
    def __init__(self, labels, batch_size, update_interval):
        self.labels = labels
        self.num_samples = len(labels)
        self.num_classes = labels.shape[1]

        class_counts = np.sum(labels, axis=0)
        class_probs = np.zeros_like(class_counts, dtype=float)
        non_zero_classes = class_counts > 0
        class_probs[non_zero_classes] = 1 / class_counts[non_zero_classes]
        class_probs /= np.sum(class_probs)
        self.class_probs = class_probs
        self.batch_size = batch_size
        self.update_interval = update_interval

    def update_probs(self, class_losses):
        # NaN이 아닌 값들만 선택
        valid_losses = class_losses[~np.isnan(class_losses)]
        
        # 유효한 손실값들의 합 계산
        total_valid_loss = np.sum(valid_losses)
        
        # 확률 계산 (NaN은 0으로 처리)
        self.class_probs = np.where(np.isnan(class_losses), 0, class_losses / total_valid_loss)
    
    def calculate_target_ratios(self, prob_ratio=0.6, min_class_count=1):
        # prob_ratio: 확률에 기반한 비율 (0.0 ~ 1.0)
        # min_class_count: 각 클래스의 최소 등장 횟수

        # 최소 클래스 카운트 적용 (원래 확률이 0인 클래스는 제외)
        base_ratios = np.where(self.class_probs > 0, min_class_count, 0)
        remaining = self.batch_size - np.sum(base_ratios)

        if remaining <= 0:
            return base_ratios  # 배치 크기가 충분하지 않으면 최소 카운트만 반환

        # 확률에 기반한 할당
        prob_based_samples = int(remaining * prob_ratio)
        prob_based_ratios = (self.class_probs * prob_based_samples).astype(int)

        # 랜덤 할당
        random_samples = remaining - prob_based_samples
        non_zero_probs = self.class_probs > 0
        random_ratios = np.zeros_like(prob_based_ratios)
        if random_samples > 0 and np.sum(non_zero_probs) > 0:
            random_ratios[non_zero_probs] = np.random.multinomial(
                random_samples, 
                self.class_probs[non_zero_probs] / np.sum(self.class_probs[non_zero_probs])
            )

        # 모든 비율 합치기
        target_ratios = base_ratios + prob_based_ratios + random_ratios

        # 남은 샘플 처리 (반올림 오차로 인해 발생할 수 있음)
        remaining = self.batch_size - np.sum(target_ratios)
        if remaining > 0:
            # 남은 샘플을 확률이 0이 아닌 클래스에 랜덤하게 할당
            additional = np.random.multinomial(
                remaining, 
                self.class_probs[non_zero_probs] / np.sum(self.class_probs[non_zero_probs]),
                size=1
            )[0]
            target_ratios[non_zero_probs] += additional

        return target_ratios
    
    def select_class(self, target_class_ratios, current_class_counts):
        # 목표 비율과 현재 카운트의 차이 계산
        diff = target_class_ratios - current_class_counts
        
        # target_class_ratios가 0보다 큰 클래스 중에서만 선택
        valid_classes = np.where((diff > 0) & (target_class_ratios > 0))[0]
        
        if len(valid_classes) > 0:
            # 유효한 클래스 중 차이가 가장 큰 클래스 선택
            return valid_classes[np.argmax(diff[valid_classes])]
        else:
            # 모든 목표가 충족되었거나 유효한 클래스가 없다면,
            # target_class_ratios가 0보다 큰 클래스 중에서만 무작위 선택
            non_zero_targets = np.where(target_class_ratios > 0)[0]
            if len(non_zero_targets) > 0:
                return np.random.choice(non_zero_targets)
            else:
                # 모든 target_class_ratios가 0이라면 None 반환
                return None
        
    def select_sample_from_class(self, selected_class, available_indices):
        # 선택된 클래스에 속하는 샘플들의 인덱스 찾기
        class_samples = [idx for idx in available_indices if self.labels[idx, selected_class] == 1]
        if class_samples:
            # 해당 클래스의 샘플 중 무작위 선택
            return np.random.choice(class_samples)
        else:
            # 해당 클래스의 샘플이 없다면 None 반환
            return None
        
    def __iter__(self):
        original_indices = np.arange(self.num_samples)
        
        for _ in range(self.update_interval):
            target_class_ratios = self.calculate_target_ratios()
            
            batch = []
            current_class_counts = np.zeros(self.num_classes)
            available_indices = original_indices.copy()
            
            while len(batch) < self.batch_size:
                selected_class = self.select_class(target_class_ratios, current_class_counts)
                if selected_class is None:
                    # 모든 클래스가 목표에 도달했다면, 남은 공간을 목표 비율에 맞게 채움
                    remaining = self.batch_size - len(batch)
                    remaining_ratios = target_class_ratios - (current_class_counts / np.sum(current_class_counts))
                    remaining_ratios = np.maximum(remaining_ratios, 0)  # 음수 비율 방지
                    remaining_ratios /= np.sum(remaining_ratios)  # 정규화
                    
                    for _ in range(remaining):
                        selected_class = np.random.choice(self.num_classes, p=remaining_ratios)
                        sample_idx = self.select_sample_from_class(selected_class, original_indices)
                        if sample_idx is None:
                            sample_idx = np.random.choice([idx for idx in original_indices if self.labels[idx, selected_class] == 1])
                        
                        batch.append(sample_idx)
                        current_class_counts += self.labels[sample_idx]
                    break
                
                sample_idx = self.select_sample_from_class(selected_class, available_indices)
                
                if sample_idx is None:
                    class_samples = [idx for idx in original_indices if self.labels[idx, selected_class] == 1]
                    if class_samples:
                        sample_idx = np.random.choice(class_samples)
                    else:
                        continue
                
                batch.append(sample_idx)
                current_class_counts += self.labels[sample_idx]
                available_indices = np.setdiff1d(available_indices, [sample_idx])
            
            self.current_class_counts = current_class_counts
        yield batch

    def __len__(self):
        return (self.num_samples + self.batch_size - 1) // self.batch_size

In [9]:
# 모델 및 데이터 설정 (이전과 동일)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = models.resnet50(pretrained=True)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, dataset.n_classes)
resnet = resnet.to(device)

optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss(reduction='none')

train_labels = dataset.train_data[1]  # 원-핫 인코딩된 레이블

batch_size = 64
total_batches = 100  # 원하는 총 배치 수
eval_interval = 1  # 평가 간격 (배치 단위)
sampler = BalancedMultiLabelSampler(train_labels,batch_size,eval_interval)
print("class_probs for sampling : ",sampler.class_probs)

total_loss = 0
all_losses = []

for batch in tqdm(range(total_batches), desc="Training Progress"):
    resnet.train()
    
    # 매 배치마다 새로운 DataLoader 생성
    trainloader = DataLoader(trainset,  batch_sampler=sampler)
    img, lbl = next(iter(trainloader))
    
    img = img.to(device)
    lbl = lbl.to(device)
    
    optimizer.zero_grad()
    
    outputs = resnet(img)
    losses = criterion(outputs, lbl)


    # sampling에 따라 loss 가중치 적용
    print('current_class_counts : ', sampler.current_class_counts)
    class_samples = torch.tensor(sampler.current_class_counts).float()
    # 가중치 계산
    weights = torch.zeros_like(class_samples, dtype=torch.float)
    non_zero_samples = class_samples > 0
    weights[non_zero_samples] = 1.0 / class_samples[non_zero_samples]
    weight_sum = weights.sum()
    weights = weights / weight_sum * torch.sum(non_zero_samples).float()
    print("loss weights : ",weights)
    losses = losses * weights.unsqueeze(0).to(device)
    loss = losses.mean()
    
    
    #loss = losses.mean()
    
    loss.backward()
    optimizer.step()
    
    total_loss += loss.item()
    all_losses.append(losses.detach().cpu().numpy())
    
    if (batch + 1) % eval_interval == 0:
        avg_loss = total_loss / eval_interval
        print(f"Batch {batch + 1}/{total_batches}, Average Loss: {avg_loss:.4f}")
        
        # 클래스별 평균 손실 계산 및 샘플러 업데이트
        batch_losses = np.concatenate(all_losses)
        
        # 분자 계산
        numerator = np.mean(batch_losses * train_labels[:len(batch_losses)], axis=0)

        # 분모 계산
        denominator = np.mean(train_labels[:len(batch_losses)], axis=0)

        # 분모가 0인 경우 결과를 0으로 설정하면서 class_losses 계산
        class_losses = np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
        
        sampler.update_probs(class_losses)
        print("class_losses : ",class_losses)
        
        # 평가 부분
        resnet.eval()
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for img, lbl in valloader:
                img = img.to(device)
                lbl = lbl.to(device)
                logits = resnet(img)    
                all_preds.append(logits)
                all_labels.append(lbl)
        all_preds = torch.cat(all_preds, dim=0)
        all_labels = torch.cat(all_labels, dim=0)
        metrics = calculate_metrics(all_preds, all_labels)
        print_metrics(metrics)
        
        # 리셋
        total_loss = 0
        all_losses = []

print("Training completed.")

class_probs for sampling :  [0.01419452 0.03431963 0.02753113 0.06507358 0.20042664 0.55674065
 0.         0.         0.05506226 0.03151362 0.01513796]


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

current_class_counts :  [ 4. 43. 34. 21. 15. 34.  0.  0. 40. 24. 43.]
loss weights :  tensor([4.1955, 0.3903, 0.4936, 0.7991, 1.1188, 0.4936, 0.0000, 0.0000, 0.4196,
        0.6993, 0.3903])
Batch 1/100, Average Loss: 0.6037
class_losses :  [3.06300151 0.2546824  0.34949961 0.66168527 0.84524104 0.44566765
 0.         0.         0.28594062 0.48483033 0.26267155]


Training Progress:   1%|          | 1/100 [00:28<47:48, 28.98s/it]

=== Overall Metrics ===
Accuracy: 0.0000
Hamming Loss: 0.3792
Subset Accuracy: 0.0000

=== Class-wise Metrics ===
Class		F1 Score	Precision	Recall
--------------------------------------------------
Class 0:	0.7184		0.5692		0.9737
Class 1:	0.3855		0.2388		1.0000
Class 2:	0.0000		0.0000		0.0000
Class 3:	0.2000		0.2500		0.1667
Class 4:	0.0000		0.0000		0.0000
Class 5:	0.0480		0.0246		1.0000
Class 8:	0.2745		0.1591		1.0000
Class 9:	0.0000		0.0000		0.0000
Class 10:	0.6733		0.5113		0.9855

=###############== Average Metrics (Active Classes) ==###############=
Macro F1: 0.2555
Macro Precision: 0.1948
Macro Recall: 0.5695
current_class_counts :  [35. 19. 17. 14. 10.  5.  0.  0. 10. 21. 33.]
loss weights :  tensor([0.3730, 0.6871, 0.7680, 0.9325, 1.3055, 2.6111, 0.0000, 0.0000, 1.3055,
        0.6217, 0.3956])
Batch 2/100, Average Loss: 0.6798
class_losses :  [0.43377769 0.76373691 0.43699041 0.55392458 0.57997754 2.74963379
 0.         0.         1.43016938 0.32055324 0.27008504]


...

=== Overall Metrics ===
Accuracy: 0.0299
Hamming Loss: 0.2415
Subset Accuracy: 0.0299

=== Class-wise Metrics ===
Class		F1 Score	Precision	Recall
--------------------------------------------------
Class 0:	0.2093		0.9000		0.1184
Class 1:	0.4615		0.3333		0.7500
Class 2:	0.5366		0.3750		0.9429
Class 3:	0.5000		0.3684		0.7778
Class 4:	0.2353		0.1818		0.3333
Class 5:	0.5000		1.0000		0.3333
Class 8:	0.3846		1.0000		0.2381
Class 9:	0.4583		0.9167		0.3056
Class 10:	0.3294		0.8750		0.2029

=###############== Average Metrics (Active Classes) ==###############=
Macro F1: 0.4017
Macro Precision: 0.6611
Macro Recall: 0.4447
current_class_counts :  [16. 35. 32. 23. 17.  5.  0.  0. 17. 18. 30.]
loss weights :  tensor([0.9828, 0.4493, 0.4914, 0.6837, 0.9250, 3.1450, 0.0000, 0.0000, 0.9250,
        0.8736, 0.5242])
Batch 99/100, Average Loss: 0.1073
class_losses :  [0.19596346 0.06670536 0.09140417 0.18613299 0.01409814 0.03061224
 0.         0.         0.02561871 0.1559138  0.19477468]


Training Progress:  99%|█████████▉| 99/100 [44:43<00:27, 27.59s/it]

=== Overall Metrics ===
Accuracy: 0.0448
Hamming Loss: 0.2144
Subset Accuracy: 0.0448

=== Class-wise Metrics ===
Class		F1 Score	Precision	Recall
--------------------------------------------------
Class 0:	0.3441		0.9412		0.2105
Class 1:	0.4828		0.3818		0.6562
Class 2:	0.5565		0.4000		0.9143
Class 3:	0.5556		0.4167		0.8333
Class 4:	0.3333		0.3333		0.3333
Class 5:	0.5000		1.0000		0.3333
Class 8:	0.5517		1.0000		0.3810
Class 9:	0.4255		0.9091		0.2778
Class 10:	0.3488		0.8824		0.2174

=###############== Average Metrics (Active Classes) ==###############=
Macro F1: 0.4554
Macro Precision: 0.6960
Macro Recall: 0.4619
current_class_counts :  [13. 39. 38. 29. 10.  8.  0.  0. 19. 24. 35.]
loss weights :  tensor([1.3542, 0.4514, 0.4633, 0.6071, 1.7605, 2.2006, 0.0000, 0.0000, 0.9266,
        0.7335, 0.5030])
Batch 100/100, Average Loss: 0.1195
class_losses :  [0.34340337 0.08214771 0.14061818 0.13830232 0.03753805 0.01936848
 0.         0.         0.24172094 0.18674333 0.13328358]


Training Progress: 100%|██████████| 100/100 [45:11<00:00, 27.11s/it]

=== Overall Metrics ===
Accuracy: 0.1119
Hamming Loss: 0.1723
Subset Accuracy: 0.1119

=== Class-wise Metrics ===
Class		F1 Score	Precision	Recall
--------------------------------------------------
Class 0:	0.4082		0.9091		0.2632
Class 1:	0.5882		0.5556		0.6250
Class 2:	0.6067		0.5000		0.7714
Class 3:	0.5098		0.3939		0.7222
Class 4:	0.4444		0.6667		0.3333
Class 5:	0.5000		1.0000		0.3333
Class 8:	0.8333		1.0000		0.7143
Class 9:	0.6182		0.8947		0.4722
Class 10:	0.4444		0.7333		0.3188

=###############== Average Metrics (Active Classes) ==###############=
Macro F1: 0.5504
Macro Precision: 0.7393
Macro Recall: 0.5060
Training completed.


In [ ]:
# 전체 프로세스 한번 정리해서 균형있게 조절해보기

In [10]:
# 평가 부분은 그대로 유지
resnet.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for img, lbl in testloader:
        img = img.to(device)
        lbl = lbl.to(device)
        logits = resnet(img)    
        score = calculate_metrics(logits, lbl)
        all_preds.append(logits)
        all_labels.append(lbl)
all_preds = torch.cat(all_preds, dim=0)
all_labels = torch.cat(all_labels, dim=0)
metrics = calculate_metrics(all_preds, all_labels)
print_metrics(metrics)

=== Overall Metrics ===
Accuracy: 0.0946
Hamming Loss: 0.1763
Subset Accuracy: 0.0946

=== Class-wise Metrics ===
Class		F1 Score	Precision	Recall
--------------------------------------------------
Class 0:	0.4231		0.7857		0.2895
Class 1:	0.5714		0.4906		0.6842
Class 2:	0.6316		0.5070		0.8372
Class 3:	0.4839		0.3488		0.7895
Class 4:	0.0000		0.0000		0.0000
Class 5:	0.0000		0.0000		0.0000
Class 8:	0.8947		1.0000		0.8095
Class 9:	0.4286		0.9000		0.2812
Class 10:	0.4330		0.8750		0.2877

=###############== Average Metrics (Active Classes) ==###############=
Macro F1: 0.4296
Macro Precision: 0.5452
Macro Recall: 0.4421


# 러닝레이트확인

# 순서변경, 난이도에 따른 샘플 선택 도입

# 난이도 확인 후 augmentation으로 공헌?